In [1]:
import gym
from gym import spaces
from gym.utils import seeding

`numpy.sign`처럼 `a`와 `b`의 크기 차이에 따라 1, 0, -1을 반환하는 함수입니다. 보상을 결정할 때 사용합니다.

In [2]:
def cmp(a, b):
    return int((a > b)) - int((a < b))

카드 덱을 표현하는 리스트입니다. 카드에는 카드가 4장씩 (다이아몬드, 하트, 스페이드, 클럽) 있기 때문에 (2카드가 5장) 비현실적인 결과가 나올 수 있습니다. 그러나 실제 카지노 블랙잭은 카드 카운팅을 방지하려고 덱 여러 벌을 같이 사용하기 때문에 큰 문제가 되지 않습니다. 그래서 교재에서도 교체를 허용하는 카드 뽑기라고 명시합니다. 만약 사실과 같은 카드 덱 한개만 사용한다면, `deck *= 4`에서 뽑은 카드를 제거하고 리스트가 비면 다시 채우도록 만듭니다.

In [3]:
# 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

카드를 한 장 그리고 두 장 뽑는 함수입니다.

In [4]:
def draw_card(np_random):
    return np_random.choice(deck)


def draw_hand(np_random):
    return [draw_card(np_random), draw_card(np_random)]

손에 에이스(1 카드)가 있고, 전체 카드 합에 10을 더해도 21 이하인지 (즉, 에이스를 11로 계산하도 게임이 끝나지 않는지) 검사합니다. 파이썬에서 특정 항목이 리스트에 들어있는지 검사할 때 `in`을 사용합니다.

In [5]:
def usable_ace(hand):  # Does this hand have a usable ace?
    return 1 in hand and sum(hand) + 10 <= 21


def sum_hand(hand):  # Return current hand total
    if usable_ace(hand):
            return sum(hand) + 10
    return sum(hand)

def is_bust(hand):  # Is this hand a bust?
    return sum_hand(hand) > 21

카드 합이 21을 초과하면 0점입니다. 이 함수의 결과를 `cmp`로 비교하여 승패 보상을 결정합니다.

In [6]:
def score(hand):  # What is the score of this hand (0 if bust)
    return 0 if is_bust(hand) else sum_hand(hand)

처음에 받은 카드가 에이스와 J/Q/K 카드(10 카드)인 경우를 내츄럴(natural)이라고 합니다. 카드 합이 21이기 때문에 상대방도 내츄럴이 아니라면 바로 승리합니다. 게임이 내츄럴 규칙을 따르는 경우 선수가 승리하면 보상이 50% 증가합니다 (즉, 1.5). 딜러의 보상은 변하지 않습니다.

In [7]:
def is_natural(hand):  # Is this hand a natural blackjack?
    return sorted(hand) == [1, 10]

전에 다룬 그리드월드는 `DiscreteEnv`를 상속했지만, 블랙잭 환경 클래스는 `Env`를 직접 상속합니다. 그래서 구현해야 할 내용이 그리드월드와 조금 다릅니다. OpenAI Gym [소스코드를](https://github.com/openai/gym/blob/master/gym/core.py) 보면, `Env`을 상속한 환경 클래스는 `_step`, `_reset`, `_render`, `_close`, `_seed`를 구현해야 합니다. 그러나 여기서는 `_step`, `_reset`, `_seed`만 구현했습니다. OpenAI Gym의 관례를 따르지 않지만, 동작은 합니다.

# 1

환경 클래스를 만드는 생성자입니다. 내츄럴 규칙을 따를지 여부를 받습니다. `spaces`는 OpenAI Gym의 관례를 따르기 위한 부분입니다.

# 2

시드를 입력받아서 새로운 시드를 반환합니다. 반환값을 직접 사용하는 대신 아래 코드와 같이 클래스 변수에 랜덤숫자생성기를 저장해두고, 뒤에서 이 생성기를 계속 사용합니다. 동일한 실험 결과가 필요한 경우 특정 시드를 반복해서 사용합니다.

# 3

OpenAI Gym `step` 함수는 그대로 `_step` 함수를 호출하고 그 (관찰, 보상, 종료여부, 기타정보) 튜플을 반환합니다. `assert` 문은 뒤의 값이 거짓이면 `AssertionError`를 내며 프로그램을 종료합니다. 보통 입력값이나 불변조건(invariant)이 유효한지 검사할 때 사용합니다.

`if action:`은 계속(hit = 1) 행동일 때 참이고, 멈추기(stick = 0) 행동일 때 거짓입니다. 주석에 설명하기 보다는 `if action == HIT:` 처럼 코드를 작성하는 편이 좋아보입니다. 선수가 카드 받기를 멈추면 딜러는 카드 합이 17 이상이 될 때까지 카드를 뽑습니다(고정 정책). 관찰(즉, 상태)은 `_get_obs` 함수를 사용하여 만듭니다. `_reset` 함수에서도 동일한 코드가 필요하기 때문에 함수로 만든 것 같습니다.

# 4

게임을 초기화합니다. 딜러와 선수 모두 카드를 두장씩 뽑고, 선수는 카드 합이 12 이상이 될 때까지 자동으로 카드를 계속 뽑습니다. 카드 합이 12 미만인 경우 계속(hit) 행동을 해도 21을 초과할 수 없기 때문에 자동으로 진행해도 됩니다.

In [8]:
class BlackjackEnv(gym.Env):
    """Simple blackjack environment
    Blackjack is a card game where the goal is to obtain cards that sum to as
    near as possible to 21 without going over.  They're playing against a fixed
    dealer.
    Face cards (Jack, Queen, King) have point value 10.
    Aces can either count as 11 or 1, and it's called 'usable' at 11.
    This game is placed with an infinite deck (or with replacement).
    The game starts with each (player and dealer) having one face up and one
    face down card.
    The player can request additional cards (hit=1) until they decide to stop
    (stick=0) or exceed 21 (bust).
    After the player sticks, the dealer reveals their facedown card, and draws
    until their sum is 17 or greater.  If the dealer goes bust the player wins.
    If neither player nor dealer busts, the outcome (win, lose, draw) is
    decided by whose sum is closer to 21.  The reward for winning is +1,
    drawing is 0, and losing is -1.
    The observation of a 3-tuple of: the players current sum,
    the dealer's one showing card (1-10 where 1 is ace),
    and whether or not the player holds a usable ace (0 or 1).
    This environment corresponds to the version of the blackjack problem
    described in Example 5.1 in Reinforcement Learning: An Introduction
    by Sutton and Barto (1998).
    https://webdocs.cs.ualberta.ca/~sutton/book/the-book.html
    """

    ###< 1 >###
    def __init__(self, natural=False):
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(32),
            spaces.Discrete(11),
            spaces.Discrete(2)))
        self._seed()

        # Flag to payout 1.5 on a "natural" blackjack win, like casino rules
        # Ref: http://www.bicyclecards.com/how-to-play/blackjack/
        self.natural = natural
        # Start the first game
        self._reset()        # Number of 
        self.nA = 2

    ###< 2 >###
    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    ###< 3 >###
    def _step(self, action):
        assert self.action_space.contains(action)
        if action:  # hit: add a card to players hand and return
            self.player.append(draw_card(self.np_random))
            if is_bust(self.player):
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        else:  # stick: play out the dealers hand, and score
            done = True
            while sum_hand(self.dealer) < 17:
                self.dealer.append(draw_card(self.np_random))
            reward = cmp(score(self.player), score(self.dealer))
            if self.natural and is_natural(self.player) and reward == 1:
                reward = 1.5
        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        return (sum_hand(self.player), self.dealer[0], usable_ace(self.player))

    ###< 4 >###
    def _reset(self):
        self.dealer = draw_hand(self.np_random)
        self.player = draw_hand(self.np_random)

        # Auto-draw another card if the score is less than 12
        while sum_hand(self.player) < 12:
            self.player.append(draw_card(self.np_random))

        return self._get_obs()